# Import termperature data

In [6]:
# 
# Import libraries and custom modules
# 
import urllib.request, urllib.parse, urllib.error
import pandas as pd
import re
import Wrangler as Wr
## Wr.extractLines
## Wr.multipleReplace
## Wr.makeDirFile


# Access URL, then request and read temperature data file 
with urllib.request.urlopen('http://berkeleyearth.lbl.gov/auto/Global/Land_and_Ocean_complete.txt') as fhand:
    # Decode file data and form a list
    file = [line.decode().strip() for line in fhand]

# Extract set: Global Average Temperature Anomaly with Sea Ice Temperature Inferred from Air Temperatures
lst = Wr.extractLines(file[77:2116])

# Parse main headers and sub headers into respective lists
h1 = Wr.multipleReplace({'%':'','-y':'Y'},file[74]).split()
h2 = re.sub('[%\s\.]*','',file[75]).split(',')

# Consolidate main and sub headers into single header list
h_list = [h1[num//2-1] + h2[num] if num > 1 else h2[num] for num in range(len(h2))]

# Convert list to DataFrame
df = pd.DataFrame(lst, columns = h_list) 

# Create Datetime Index
df['Date'] = pd.to_datetime(df[['Year', 'Month']].assign(DAY=1))
df = df.set_index('Date').to_period('M')

# Set data to numeric
df[h_list[2:]] = df[h_list[2:]].apply(pd.to_numeric, errors='coerce')

# Export csv file
fullname = Wr.makeDirFile('Temperature')
export_csv = df.to_csv(fullname, index='Date', header=True)